In [74]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3

from collections import Counter, defaultdict
from bs4 import BeautifulSoup

import numpy as np
from numpy import random

In [203]:
#load data

with sqlite3.connect('../data/raw/database.sqlite') as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    matches = pd.read_sql_query("SELECT * from Match", con, parse_dates=['date'])
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)
    players = pd.read_sql_query("SELECT * from Player", con)
    players_stats = pd.read_sql_query("SELECT * from Player_Stats", con)

In [204]:
selected_countries = ['England']
countries = countries[countries.name.isin(selected_countries)]
leagues = countries.merge(leagues, on='id', suffixes=('', '_y'))
matches = matches[matches.league_id.isin(leagues.id)]
matches.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
1728,1729,1729,1729,2008/2009,1,2008-08-17,489042,10260,10261,1,...,10.00,1.28,5.5,12.00,1.30,4.75,10.0,1.29,4.50,11.00
1729,1730,1729,1729,2008/2009,1,2008-08-16,489043,9825,8659,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1730,1731,1729,1729,2008/2009,1,2008-08-16,489044,8472,8650,0,...,1.73,5.50,3.8,1.65,5.00,3.40,1.7,4.50,3.40,1.73
1731,1732,1729,1729,2008/2009,1,2008-08-16,489045,8654,8528,2,...,3.75,1.90,3.5,4.35,1.91,3.25,4.0,1.91,3.25,3.80
1732,1733,1729,1729,2008/2009,1,2008-08-17,489046,10252,8456,4,...,3.75,1.90,3.5,4.35,1.91,3.25,4.0,1.91,3.30,3.75


In [206]:
def get_team_long_name(team_id):
    return teams.loc[teams.team_api_id == team_id, 'team_long_name'].values[0]

def get_team_short_name(team_id):
    return teams.loc[teams.team_api_id == team_id, 'team_short_name'].values[0]

def get_formation(match_api_id, team_type):
    formation_dict = Counter([matches.loc[matches.match_api_id == match_api_id]['%s_player_Y%d'%(team_type, i)].values[0] for i in range(1, 12)])
    sorted_keys = sorted(formation_dict)

    formation = ''
    for key in sorted_keys[1:-1]:
        y = formation_dict[key]
        formation += '%d-' % y
    formation += '%d' % formation_dict[sorted_keys[-1]]
    
    return formation

matches['home_team_long_name'] = matches.home_team_api_id.map(get_team_long_name)
matches['home_team_short_name'] = matches.home_team_api_id.map(get_team_short_name)

matches['away_team_long_name'] = matches.away_team_api_id.map(get_team_long_name)
matches['away_team_short_name'] = matches.away_team_api_id.map(get_team_short_name)

matches['home_team_formation'] = matches.match_api_id.apply(get_formation, args=('home', ))
matches['away_team_formation'] = matches.match_api_id.apply(get_formation, args=('away',))

In [207]:
matches[['home_team_goal', 'away_team_goal', 'home_team_short_name', 'away_team_short_name']].head()

,home_team_goal,away_team_goal,home_team_short_name,away_team_short_name
1728,1,1,MUN,NEW
1729,1,0,ARS,WBA
1730,0,1,SUN,LIV
1731,2,1,WHU,WIG
1732,4,2,AVL,MCI


In [208]:
def calculate_result(match_df):
    home_team_goals = match_df['home_team_goal']
    away_team_goals = match_df['away_team_goal']
    
    if home_team_goals > away_team_goals:
        return 'home'
    elif away_team_goals > home_team_goals:
        return 'away'
    else:
        return 'draw'

matches['result'] = matches.apply(calculate_result, axis=1)

In [209]:
def calculate_crosses_count(crosses):
    parsed_xml = BeautifulSoup(crosses, 'xml')
    num_crosses = defaultdict(int)
    
    for x in parsed_xml.find_all('value'):
        if x.crosses:
            if x.team:
                num_crosses[x.team.text] += 1
    
    return num_crosses

crosses_info = matches.cross.map(calculate_crosses_count)

In [210]:
def get_crosses_list(crosses_info, matches):
    home_team_crosses = []
    away_team_crosses = []
    counter = 0
    
    for team_id in matches.home_team_api_id.values:
        home_team_crosses.append(crosses_info.iloc[counter][str(team_id)])
        counter += 1
        
    counter = 0
    for team_id in matches.away_team_api_id.values:
        away_team_crosses.append(crosses_info.iloc[counter][str(team_id)])
        counter += 1
        
    return (home_team_crosses, away_team_crosses)

In [211]:
matches['num_home_team_crosses'], matches['num_away_team_crosses'] = get_crosses_list(crosses_info, matches)

In [212]:
features = ['date', 'home_team_formation', 'home_team_short_name', 'away_team_formation', \
            'away_team_short_name',  'stage', 'num_home_team_crosses', 'num_away_team_crosses',\
            'B365H','B365D','B365A','result']

In [213]:
from sklearn.preprocessing import LabelEncoder

In [214]:
for feature in ['home_team_formation', 'home_team_short_name', 'away_team_formation', 'away_team_short_name', 'result']:
    lbl = LabelEncoder()
    matches[feature] = lbl.fit_transform(matches[feature])

In [215]:
matches['weekday'] = matches.date.dt.weekday
matches['day'] = matches.date.dt.day

In [221]:
matches['B365H'] = matches.B365H.fillna(-999)
matches['B365A'] = matches.B365A.fillna(-999)
matches['B365D'] = matches.B365D.fillna(-999)

In [222]:
X = matches[['home_team_formation', 'home_team_short_name', 'away_team_formation', \
            'away_team_short_name',  'stage', 'num_home_team_crosses', 'num_away_team_crosses', \
            'B365H','B365D','B365A', 'weekday', 'day']]
y = matches.result

In [223]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

In [231]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

est = LogisticRegression()
est.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [232]:
from sklearn.metrics import accuracy_score

print('Accuracy score on test set %f '%(accuracy_score(y_test, est.predict(X_test))))

Accuracy score on test set 0.581395 
